### INST447 Project Part 2
#### Members: Alexander Chui and Matthew Chan

### What factors affect ratings for an anime?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import numpy as np
from requests import get

In [3]:
#libraries imported for handling rate limits and console formatting
from time import time, sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [4]:
import regex as re

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis

import nltk; 

C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#Connecting to the URL on the web to access its content (Crunchyroll)
# html=urlopen('https://www.crunchyroll.com/fullmetal-alchemist-brotherhood/reviews/helpful/page1')
# html.status

In [7]:
#Connecting to the URL on the web to access its content (MAL)
html=urlopen('https://myanimelist.net/anime/1/Cowboy_Bebop/reviews?p=1')
html.status

200

In [8]:
#Creating an instance of beautifulsoup object to begin web scraping
bs=BeautifulSoup(html.read(),'html.parser')

In [9]:
#Acessing the container objects with the anime info 
anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})
print(len(anime_review_containers))

20


In [10]:
#Creating empty lists to store all the items 
review = []

In [11]:
# Preparing the monitoring of the loop
start_time = time()
request = 0

In [12]:
#Extracting each job name in loop
for page in range(1,22):
    # Make a get request
        #https://myanimelist.net/anime/genre/1/Action?page=1
        #https://myanimelist.net/anime/genre/1/Action?page=2
        response = get('https://myanimelist.net/anime/1/Cowboy_Bebop/reviews?p='+ str(page))
        
#         # Pause the loop
#         sleep(randint(8,15))

#         # Monitor the requests
#         request += 1
#         elapsed_time = time() - start_time
#         print('Request:{}; Frequency: {} requests/s'.format(request, 
#                                                             request/elapsed_time))
#         clear_output(wait = True)

#         # Throw a warning for non-200 status codes
#         if response.status_code != 200:
#             warn('Request: {}; Status code: {}'.format(request, 
#                                                        response.status_code))

#         # Break the loop if the number of requests is greater than expected
#         if request > 100:
#             warn('Number of requests was greater than expected.')
#             break
        
        # Parse the content of the request with BeautifulSoup
        bs = BeautifulSoup(response.text, 'html.parser')
        
        #Acessing the container objects with the anime info 
        anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})

        #Extracting all the elements 
        for container in anime_review_containers:
        # If the movie has Metascore, then extract:
        #  if container.find('div', {'class':{'scoremem'}})  is not None:
        
        # The name of anime
            try: 
                reviews = container.find('div', {'class':'spaceit textReadability word-break pt8 mt8'})
                review.append(reviews.text)
            except:
                review.append('Missing')

In [13]:
#Creating a pandas dataframe to store the objects extracted
anime_review = pd.DataFrame({'Review': review
})
print(anime_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  420 non-null    object
dtypes: object(1)
memory usage: 3.4+ KB
None


In [14]:
# Display the top 5 rows of dataset
anime_review.head()

,Review
0,\n\n\n\nOverall\n10\n\n\nStory\n10\n\n\nAnimat...
1,\n\n\n\nOverall\n7\n\n\nStory\n5\n\n\nAnimatio...
2,\n\n\n\nOverall\n6\n\n\nStory\n6\n\n\nAnimatio...
3,\n\n\n\nOverall\n9\n\n\nStory\n8\n\n\nAnimatio...
4,\n\n\n\nOverall\n10\n\n\nStory\n0\n\n\nAnimati...


In [15]:
# Convert to list
data = anime_review['Review'].tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

print(data[0])

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-b412c0db5fd7>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


 Overall 10 Story 10 Animation 9 Sound 10 Character 10 Enjoyment 9 People who know me know that I'm not a fan of episodic anime series unless they're either one season (12-14 episodes) long or a slice of life series. Why? They have a tendency to get boring, or repetitive. But all rules and preferences have exceptions. You know what? Cowboy Bebop is that exception. The story is set in a space western setting - a genre and setting I'm loving more and more for each show I watch that falls under the genre. We follow two bounty hunters, Spike and Jet, who own a ship called the Bebop. They travel the Solar system, chasing wanted criminals to earn money. Along the way, they also pick up two women; the debt-laden Faye Valentine and the playful kid and computer genius Edward (yes, Ed&rsquo;s a girl). Each episode brings about a new bounty which they chase after, and while that doesn't sound too exciting to watch 26 episodes in a row, you'll end up loving the show. All the different events makes

In [16]:
# Clean and tokenize the reviews

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'people', 'who', 'know', 'me', 'know', 'that', 'not', 'fan', 'of', 'episodic', 'anime', 'series', 'unless', 'they', 're', 'either', 'one', 'season', 'episodes', 'long', 'or', 'slice', 'of', 'life', 'series', 'why', 'they', 'have', 'tendency', 'to', 'get', 'boring', 'or', 'repetitive', 'but', 'all', 'rules', 'and', 'preferences', 'have', 'exceptions', 'you', 'know', 'what', 'cowboy', 'bebop', 'is', 'that', 'exception', 'the', 'story', 'is', 'set', 'in', 'space', 'western', 'setting', 'genre', 'and', 'setting', 'loving', 'more', 'and', 'more', 'for', 'each', 'show', 'watch', 'that', 'falls', 'under', 'the', 'genre', 'we', 'follow', 'two', 'bounty', 'hunters', 'spike', 'and', 'jet', 'who', 'own', 'ship', 'called', 'the', 'bebop', 'they', 'travel', 'the', 'solar', 'system', 'chasing', 'wanted', 'criminals', 'to', 'earn', 'money', 'along', 'the', 'way', 'they', 'also', 'pick', 'up', 'two', 'women', 'the', 'debt', 'laden', 

In [17]:
# Create bigrams, remove stop words and lemmatize the reviews

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'cowboy', 'bebop', 'was', 'one', 'of', 'the', 'first', 'anime', 'that', 'truly', 'impressed', 'me', 'one', 'of', 'the', 'first', 'anime', 'that', 'convinced', 'me', 'the', 'anime', 'genre', 'as', 'whole', 'had', 'something', 'going', 'for', 'it', 'that', 'anime', 'has', 'qualities', 'that', 'separate', 'it', 'from', 'other', 'forms', 'of', 'entertainment', 'it', 'is', 'show', 'that', 'has', 'withstood', 'the', 'test', 'of', 'time', 'and', 'is', 'often', 'viewed', 'as', 'masterpiece', 'by', 'many', 'with', 'such', 'popularity', 'and', 'praise', 'there', 'is', 'bound', 'to', 'be', 'group', 'of', 'the', 'opposite', 'extreme', 'that', 'dislike', 'the', 'show', 'now', 'bear', 'in', 'my', 'mind', 'that', 'although', 'this', 'group', 'is', 'minority', 'their', 'points', 'are', 'valid', 'as', 'reviewer', 'look', 'at', 'all', 'angles', 'and', 'try', 'my', 'best', 'to', 'keep', 'bias', 'at', 'an', 'all', 'time', 'low', 'having'

In [18]:
# Defining removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Defining making bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Defining lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'people', 'know', 'know', 'fan', 'episodic', 'anime', 'series', 'season', 'episode', 'long', 'slice', 'life', 'series', 'tendency', 'get', 'bore', 'repetitive', 'rule', 'preference', 'exception', 'know', 'cowboy', 'bebop', 'exception', 'story', 'set', 'space', 'western', 'set', 'genre', 'set', 'love', 'show', 'watch', 'fall', 'genre', 'follow', 'bounty_hunter', 'spike', 'jet', 'ship', 'call', 'bebop', 'travel', 'chasing', 'want', 'criminal', 'earn', 'money', 'way', 'also', 'pick', 'woman', 'debt', 'laden', 'faye_valentine', 'playful', 'kid', 'computer', 'genius', 'edward', 'rsquo', 'girl', 'episode', 'bring', 'new', 'bounty', 'chase', 'sound', 'exciting', 'watch', 'episode', 'row', 'end', 'loving', 'show', 'different', 'event', 'make', 'certain', 'degree', 'sometimes', 'wonder', 'thing', 'end', 'however', 'alone', 'enough', 'give', 'story', 'rating', 'give', 'give', 'rating', 'let', 'continue', 'thing', 'elevate', 'sh

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 6), (2, 1), (3, 1), (4, 1), (5, 2), (6, 4), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 7), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 2), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 3), (45, 1), (46, 4), (47, 1), (48, 1), (49, 8), (50, 1), (51, 1), (52, 2), (53, 1), (54, 2), (55, 2), (56, 1), (57, 3), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 3), (71, 3), (72, 1), (73, 6), (74, 1), (75, 4), (76, 1), (77, 1), (78, 2), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 3), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 3), (101, 1), (102, 1), (103, 2), (104, 2), (105, 2), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1),

In [21]:
# Using LDA to develop a topic model with 8 models

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 8 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.000*"bebop" + 0.000*"cowboy" + 0.000*"episode" + 0.000*"anime" + 0.000*"story" + 0.000*"show" + 0.000*"character" + 0.000*"good" + 0.000*"watch" + 0.000*"spike"'), (1, '0.010*"simplistic" + 0.002*"sentence" + 0.001*"crappy" + 0.001*"fortunately" + 0.001*"quintet" + 0.001*"unlucky" + 0.001*"engine" + 0.001*"prey" + 0.001*"punchy" + 0.001*"prestigious"'), (2, '0.000*"bebop" + 0.000*"anime" + 0.000*"character" + 0.000*"cowboy" + 0.000*"series" + 0.000*"show" + 0.000*"even" + 0.000*"time" + 0.000*"well" + 0.000*"story"'), (3, '0.009*"spike" + 0.007*"crew" + 0.006*"past" + 0.006*"day" + 0.005*"matter" + 0.005*"vicious" + 0.005*"become" + 0.005*"individual" + 0.005*"support" + 0.005*"long"'), (4, '0.027*"character" + 0.022*"anime" + 0.020*"bebop" + 0.020*"show" + 0.017*"story" + 0.016*"cowboy" + 0.015*"episode" + 0.012*"watch" + 0.011*"series" + 0.010*"good"'), (5, '0.003*"fu" + 0.003*"kung" + 0.001*"pocky" + 0.001*"freezer" + 0.001*"fulfil" + 0.001*"weirdo" + 0.001*"mario" + 0.001*"

In [22]:
# Generate a visualization of the topic models

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.341237  0.001554       1        1  94.724864
3     -0.059874  0.206451       2        1   4.341001
6     -0.079011 -0.086737       3        1   0.644430
7     -0.047259 -0.031773       4        1   0.197090
1     -0.040328 -0.023906       5        1   0.040662
5     -0.039559 -0.023275       6        1   0.040548
2     -0.037604 -0.021157       7        1   0.005712
0     -0.037604 -0.021157       8        1   0.005693, topic_info=              Term        Freq       Total Category  logprob  loglift
158          spike  567.000000  567.000000  Default  30.0000  30.0000
118           past  276.000000  276.000000  Default  29.0000  29.0000
89             jet  288.000000  288.000000  Default  28.0000  28.0000
466           seem  233.000000  233.000000  Default  27.0000  27.0000
261           crew  160.000000  160.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
16   bounty_hunter    0.000733  139.013649   Topic8  -8.9343  -2.3790
19            cast    0.000733  141.388656   Topic8  -8.9343  -2.3959
37       different    0.000733  248.206843   Topic8  -8.9343  -2.9587
46             end    0.000733  420.556259   Topic8  -8.9343  -3.4860
47       enjoyment    0.000733  613.263885   Topic8  -8.9343  -3.8632

[395 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
7463      2  0.920896   accurately
1796      2  1.023216  acknowledge
548       1  0.889679      already
548       2  0.110063      already
1         1  0.999153         also
...     ...       ...          ...
181       1  0.997317          way
181       2  0.002645          way
5203      3  0.969471       weapon
528       1  0.999330         well
3456      2  0.997685    willingly

[234 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 7, 8, 2, 6, 3, 1])